# PySpark Warmup

## Installation for PySpark




In [ ]:
!apt-get -y install openjdk-8-jre-headless
!pip install pyspark==3.0.1

#### Start a simple Spark Session

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType,StructType,IntegerType,StructField
spark = SparkSession.builder.appName('Lab3').getOrCreate()

Data Schema

In [3]:
data_schema = [StructField('age',IntegerType(),True),
              StructField('name',StringType(),True)]
final_struc = StructType(fields=data_schema)


Load the people.json gile, have Spark infer the data types.

In [4]:
df = spark.read.json('people.json',schema=final_struc)

#### What are the column names?

In [5]:
df.columns

['age', 'name']

#### What is the schema?

In [6]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



Show whole DataFrame 

In [7]:
df.show()


+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



Print out the first 2 rows.

In [11]:
# Didn't strictly need a for loop, could have just then head()
for row in df.head(2):
    print(row)

Row(age=None, name='Michael')
Row(age=30, name='Andy')


Use describe() to learn about the DataFrame

In [12]:
df.describe()

DataFrame[summary: string, age: string, name: string]

Use another data frame to learn about the statistical report

In [13]:
temp = df.describe()
temp.show()

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   null|
| stddev|7.7781745930520225|   null|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



Get the mean of age directly

In [16]:
from pyspark.sql.functions import mean
df.select(mean("age")).show()

+--------+
|avg(age)|
+--------+
|    24.5|
+--------+



What is the max and min of the Volume column?

In [17]:
# Could have also used describe
from pyspark.sql.functions import max,min
df.select(max("age"),min("age")).show()

+--------+--------+
|max(age)|min(age)|
+--------+--------+
|      30|      19|
+--------+--------+



How many days was the age smaller than 30?

In [18]:
df.filter("age < 30").count()

1

In [19]:
from pyspark.sql.functions import count
result = df.filter(df['age'] < 30)
result.select(count('age')).show()

+----------+
|count(age)|
+----------+
|         1|
+----------+



# PySpark Exercise

Let's get some quick practice with your new Spark DataFrame skills, you will be asked some basic questions about some stock market data, in this case Walmart Stock from the years 2012-2017. This exercise will just ask a bunch of questions, unlike the future machine learning exercises, which will be a little looser and be in the form of "Consulting Projects", but more on that later!

For now, just answer the questions and complete the tasks below.

#### [Hint](https://spark.apache.org/docs/3.0.1/api/python/index.html)

#### Use the walmart_stock.csv file to Answer and complete the  tasks below!

#### Load the Walmart Stock CSV File, have Spark infer the data types.

In [20]:
df = spark.read.csv('/content/walmart_stock.csv',header=True,inferSchema=True)

#### What are the column names?

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

#### What does the Schema look like?

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



## Checkpoint 1:Print out the first 5 columns.

Row(Date='2012-01-03', Open=59.970001, High=61.060001, Low=59.869999, Close=60.330002, Volume=12668800, Adj Close=52.619234999999996)
Row(Date='2012-01-04', Open=60.209998999999996, High=60.349998, Low=59.470001, Close=59.709998999999996, Volume=9593300, Adj Close=52.078475)
Row(Date='2012-01-05', Open=59.349998, High=59.619999, Low=58.369999, Close=59.419998, Volume=12768200, Adj Close=51.825539)
Row(Date='2012-01-06', Open=59.419998, High=59.450001, Low=58.869999, Close=59.0, Volume=8069400, Adj Close=51.45922)
Row(Date='2012-01-09', Open=59.029999, High=59.549999, Low=58.919998, Close=59.18, Volume=6679300, Adj Close=51.616215000000004)


## Checkpoint 2: Use describe() to learn about the DataFrame.

+-------+----------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|      Date|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+----------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|      1258|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean|      null| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|      null|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|2012-01-03|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|2016-12-30|         90.800003|        90.970001|            89.25|        90.4700


#### There are too many decimal places for mean and stddev in the describe() dataframe. Format the numbers to just show up to two decimal places. Pay careful attention to the datatypes that .describe() returns, we didn't cover how to do this exact formatting, but we covered something very similar. [Check this link for a hint](https://spark.apache.org/docs/3.0.1/api/python/pyspark.sql.html?highlight=describe#pyspark.sql.DataFrame.describe)

In [31]:
# Uh oh Strings! 
df.describe().printSchema()

root
 |-- summary: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- Adj Close: string (nullable = true)



In [32]:
from pyspark.sql.functions import format_number

In [33]:
# hint
result = df.describe()
result.select(result['summary'],
              format_number(result['Open'].cast('float'),2).alias('Open'),
              format_number(result['Close'].cast('float'),2).alias('Close'),
             ).show()

+-------+--------+--------+
|summary|    Open|   Close|
+-------+--------+--------+
|  count|1,258.00|1,258.00|
|   mean|   72.36|   72.39|
| stddev|    6.77|    6.76|
|    min|   56.39|   56.42|
|    max|   90.80|   90.47|
+-------+--------+--------+



## Checkpoint 3: format number

+-------+--------+--------+--------+--------+--------+
|summary|    Open|    High|     Low|   Close|  Volume|
+-------+--------+--------+--------+--------+--------+
|  count|1,258.00|1,258.00|1,258.00|1,258.00|    1258|
|   mean|   72.36|   72.84|   71.92|   72.39| 8222093|
| stddev|    6.77|    6.77|    6.74|    6.76| 4519780|
|    min|   56.39|   57.06|   56.30|   56.42| 2094900|
|    max|   90.80|   90.97|   89.25|   90.47|80898100|
+-------+--------+--------+--------+--------+--------+



## Checkpoint 4: HV Ratio
HV Ratio = df["High"]/df["Volume"]
#### Create a new dataframe with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day.

+--------------------+
|            HV Ratio|
+--------------------+
|4.819714653321546E-6|
|6.290848613094555E-6|
|4.669412994783916E-6|
|7.367338463826307E-6|
|8.915604778943901E-6|
|8.644477436914568E-6|
|9.351828421515645E-6|
| 8.29141562102703E-6|
|7.712212102001476E-6|
|7.071764823529412E-6|
|1.015495466386981E-5|
|6.576354146362592...|
| 5.90145296180676E-6|
|8.547679455011844E-6|
|8.420709512685392E-6|
|1.041448341728929...|
|8.316075414862431E-6|
|9.721183814992126E-6|
|8.029436027707578E-6|
|6.307432259386365E-6|
+--------------------+
only showing top 20 rows



#### What day had the Peak High in Price?

In [ ]:
# Didn't need to really do this much indexing
# Could have just shown the entire row
df.orderBy(df["High"].desc()).head(1)[0][0]

datetime.datetime(2015, 1, 13, 0, 0)

## Checkpoint 5: What is the mean, max and min of the Close column?

In [ ]:
# Could have also used describe
from pyspark.sql.functions import mean,max,min

+-----------------+----------+----------+
|       avg(Close)|max(Close)|min(Close)|
+-----------------+----------+----------+
|72.38844998012726| 90.470001| 56.419998|
+-----------------+----------+----------+



## Checkpoint 6: How many days was the Close lower than 60 dollars?

In [ ]:
from pyspark.sql.functions import count # hint


+------------+
|count(Close)|
+------------+
|          81|
+------------+



## Checkpoint 7: What percentage of the time was the High greater than 80 dollars ?
#### In other words, (Number of Days High>80)/(Total Days in the dataset)

In [ ]:
# Many ways to do this


9.141494435612083

## Checkpoint 8: What is the Pearson correlation between High and Volume?
hint: corr("High","Volume")
#### [Hint](https://spark.apache.org/docs/3.0.1/api/python/pyspark.sql.html?highlight=dataframestatfunctions#pyspark.sql.DataFrameStatFunctions)

In [ ]:
from pyspark.sql.functions import corr # hint


+-------------------+
| corr(High, Volume)|
+-------------------+
|-0.3384326061737161|
+-------------------+



#### What is the max High per year?

In [34]:
from pyspark.sql.functions import year
yeardf = df.withColumn("Year",year(df["Date"]))

In [35]:
max_df = yeardf.groupBy('Year').max()
max_df.select('Year','max(High)').show()

+----+---------+
|Year|max(High)|
+----+---------+
|2015|90.970001|
|2013|81.370003|
|2014|88.089996|
|2012|77.599998|
|2016|75.190002|
+----+---------+



## Checkpoint 9: What is the average Close for each Calendar Month?
#### In other words, across all the years, what is the average Close price for Jan,Feb, Mar, etc... Your result will have a value for each of these months. 

In [27]:
from pyspark.sql.functions import month 
# hint
monthdf = df.withColumn("Month",month("Date"))

In [28]:
# hint: group by "Month"


+-----+-----------------+
|Month|       avg(Close)|
+-----+-----------------+
|    1|71.44801958415842|
|    2|  71.306804443299|
|    3|71.77794377570092|
|    4|72.97361900952382|
|    5|72.30971688679247|
|    6| 72.4953774245283|
|    7|74.43971943925233|
|    8|73.02981855454546|
|    9|72.18411785294116|
|   10|71.57854545454543|
|   11| 72.1110893069307|
|   12|72.84792478301885|
+-----+-----------------+



# Great!